In [ ]:
# !pip install openpipe-art==0.5.0 langchain-core tenacity datasets vllm faiss-cpu chromadb requests lxml numpy transformers torch gql==3.4.1 peft 
# !pip install langchain-core tenacity datasets vllm

# ============================================================
# PATH SETUP - Add project root to sys.path
# ============================================================
import sys
from pathlib import Path

# Navigate up to project root from training/serverless/
PROJECT_ROOT = Path().resolve().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Project root: {PROJECT_ROOT}")

In [ ]:
import os
from secretsConfig import oaiKey, wandbKey, openRouterKey

# Set environment variables for API keys
os.environ["OPENAI_API_KEY"] = oaiKey
os.environ["WANDB_API_KEY"] = wandbKey
os.environ["OPENROUTER_API_KEY"] = openRouterKey

# Validate keys
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY is required for RULER functionality.")

if not os.environ.get("WANDB_API_KEY"):
    raise ValueError("WANDB_API_KEY is required for W&B.")

if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("OPENROUTER_API_KEY is required for Gemini judge.")

print("✅ API keys loaded")

In [ ]:
# Import centralized paths
from config.paths import LEGAL_XML_FILE, CHROMA_DB_PATH

# Import RAG tools
from rag_tools.semantic_search import FAISSSemanticSearch
from rag_tools.keyword_search import keyword_search
from rag_tools.read_document import read_document_part

# Wrap tools with error handling
def keyword_search_tool(query: str, num: int = 5) -> str:
    """BM25 keyword search"""
    try:
        return keyword_search(str(LEGAL_XML_FILE), query, num)
    except Exception as e:
        error_msg = f"[TOOL ERROR] keyword_search failed: {str(e)}"
        print(error_msg)
        return error_msg

def semantic_search_tool(query: str, num: int = 5) -> str:
    """FAISS semantic search"""
    try:
        searcher = FAISSSemanticSearch(chroma_path=str(CHROMA_DB_PATH))
        return searcher.search(query, num)
    except Exception as e:
        error_msg = f"[TOOL ERROR] semantic_search failed: {str(e)}"
        print(error_msg)
        return error_msg

def read_document_part_tool(part_id: str) -> str:
    """Read document part by ID"""
    try:
        if " " in part_id or len(part_id) > 100:
            error_msg = f"[INVALID PART_ID] '{part_id[:50]}...' is not a valid part_id format."
            print(f"[ERROR] Model tried to read invalid part_id: {part_id[:50]}...")
            return error_msg
        return read_document_part(str(LEGAL_XML_FILE), part_id)
    except Exception as e:
        error_msg = f"[TOOL ERROR] Failed to read document part: {str(e)}"
        print(f"[ERROR] {error_msg}")
        return error_msg

print("✅ RAG tools loaded with centralized paths")
print(f"   - Legal XML: {LEGAL_XML_FILE}")
print(f"   - ChromaDB: {CHROMA_DB_PATH}")

In [ ]:
from dotenv import load_dotenv
import random

import art
from art.serverless.backend import ServerlessBackend

# Import serverless configuration
from config.training_config import BASE_MODEL, WANDB_CONFIG
import config_serverless

load_dotenv()
random.seed(42)

# Print configuration
config_serverless.print_config()

# Get model config from serverless config
model_config = config_serverless.get_model_config()
print(f"📦 Creating model: {model_config['base_model']}")

# Create trainable model
model = art.TrainableModel(**model_config)

# Initialize serverless backend
backend = config_serverless.get_backend()
print(f"🔧 Backend: W&B Serverless")

# Register the model
await model.register(backend)
print(f"✅ Model registered\n")

In [ ]:
from textwrap import dedent
from pydantic import BaseModel, Field
from openai import AsyncOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
import art

# Import MAX_TURNS from config
from config.training_config import MAX_TURNS


class FinalAnswer(BaseModel):
    answer: str
    source_ids: list[str]


class LegalScenario(BaseModel):
    id: str
    question: str
    gold_answer: str | None = None
    gold_part_ids: list[str] | None = None


class LegalScenarioStep(BaseModel):
    step: int
    scenario: LegalScenario


async def rollout(model: art.Model, legal_scenario_step: LegalScenarioStep) -> art.Trajectory:
    """Execute one trajectory rollout"""
    scenario = legal_scenario_step.scenario
    
    traj = art.Trajectory(
        reward=0.0,
        messages_and_choices=[],
        metadata={"scenario_id": scenario.id, "step": legal_scenario_step.step},
    )

    system_prompt = dedent(
        f"""
        You are a legal research assistant that can search legal documents to answer questions.

        You have access to the following tools:
        - search_keyword(query: str, num: int) -> str
        - search_semantic(query: str, num: int) -> str
        - read_document_part(part_id: str) -> str

        You may call one tool per turn, for up to {MAX_TURNS} turns.

        When ready, give your final answer in this format:
        <answer>
        [your answer or "I don't know" if insufficient information]
        <sources>
        <source>doc_id</source>
        </sources>
        </answer>
        """
    )

    traj.messages_and_choices = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": scenario.question},
    ]

    # Define tools using centralized tool functions
    def return_final_answer(answer: str, source_ids: list[str]) -> FinalAnswer:
        return FinalAnswer(answer=answer, source_ids=source_ids)

    tools = [keyword_search_tool, semantic_search_tool, read_document_part_tool, return_final_answer]
    tools_by_name = {t.__name__: t for t in tools}
    traj.tools = [convert_to_openai_tool(t) for t in tools]

    client = AsyncOpenAI(
        base_url=model.inference_base_url,
        api_key=model.inference_api_key,
    )

    for _ in range(MAX_TURNS):
        response = await client.chat.completions.create(
            model=model.get_inference_name(),
            temperature=1,
            messages=traj.messages(),
            tools=traj.tools,
        )

        response_message = response.choices[0].message
        traj.messages_and_choices.append(response.choices[0])

        if not response_message.tool_calls:
            return traj

        try:
            import json
            for tool_call in response_message.tool_calls:
                tool_name = tool_call.function.name
                if tool_name in tools_by_name:
                    tool_args = json.loads(tool_call.function.arguments)
                    result = tools_by_name[tool_name](**tool_args)
                    traj.messages_and_choices.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": tool_name,
                        "content": str(result),
                    })

                    if tool_name == "return_final_answer":
                        return traj
        except Exception as e:
            print(f"Tool error: {e}")
            return traj

    return traj


print("✅ Rollout function defined!")

In [ ]:
import json
import os
from litellm import acompletion

# Import centralized data path
from config.paths import TRAINING_DATA_FILE
from config.training_config import MAX_TURNS

# Load training data from centralized path
print(f"📂 Loading data from {TRAINING_DATA_FILE}...")
with open(TRAINING_DATA_FILE, 'r') as f:
    data = json.load(f)

# Convert to LegalScenario objects
training_scenarios = []
for item in data.get("items", []):
    for row in item.get("rows", []):
        training_scenarios.append(
            LegalScenario(
                id=str(row["row_index"]),
                question=row["question"],
                gold_answer=row.get("model_answer", ""),
                gold_part_ids=row.get("sources", [])
            )
        )

print(f"✅ Loaded {len(training_scenarios)} scenarios\n")


# Gemini judge with updated scoring criteria
async def gemini_judge(group: art.TrajectoryGroup) -> art.TrajectoryGroup:
    """Score trajectories with new criteria"""
    trajectories = group.trajectories
    
    if len(trajectories) <= 1:
        for traj in trajectories:
            traj.reward = 0.0
        return group

    # Analyze trajectories
    analyses = []
    for i, traj in enumerate(trajectories):
        messages = traj.messages()
        final_answer = messages[-1].get("content", "") if messages else ""

        format_errors = []
        num_searches = 0
        num_turns = sum(1 for m in messages if isinstance(m, dict) and m.get("role") == "assistant")

        for msg in messages:
            if isinstance(msg, dict) and msg.get("role") == "tool":
                content = msg.get("content", "")
                if "search" in msg.get("name", "").lower():
                    num_searches += 1
                if any(err in content for err in ["[TOOL ERROR]", "[INVALID PART_ID]", "[PART NOT FOUND]"]):
                    format_errors.append(content[:80])

        analyses.append({
            "index": i,
            "final_answer": final_answer[:400],
            "num_turns": num_turns,
            "num_searches": num_searches,
            "has_format_error": len(format_errors) > 0,
        })

    # Build judge prompt
    analysis_text = "\\n".join([
        f"**Response {a['index']+1}:** {a['final_answer']}\\n"
        f"  Turns: {a['num_turns']}/{MAX_TURNS}, Searches: {a['num_searches']}, "
        f"Errors: {'YES' if a['has_format_error'] else 'NO'}"
        for a in analyses
    ])

    judge_prompt = f"""Score {len(trajectories)} legal research responses:

RULES:
- Correct answer: 1.0 to 2.0
- "I don't know": 0.0 to 1.0 (GOOD - avoids hallucination)
- Wrong answer: -1.0 to 0.0
- Format errors: -2.0 to -1.0

{analysis_text}

Return JSON: {{"scores": [{{"base_score": X, "reasoning": "..."}}, ...]}}"""

    try:
        response = await acompletion(
            model="openrouter/google/gemini-2.5-flash",
            messages=[{"role": "user", "content": judge_prompt}],
            api_key=os.environ["OPENROUTER_API_KEY"],
            max_tokens=500,
        )

        import re
        result_text = response.choices[0].message.content.strip()
        json_match = re.search(r'\\{[\\s\\S]*\\}', result_text)

        if json_match:
            result = json.loads(json_match.group())
            base_scores = [item["base_score"] for item in result["scores"]]
        else:
            base_scores = [0.0] * len(trajectories)

        # Apply efficiency bonuses
        final_scores = []
        for base_score, analysis in zip(base_scores, analyses):
            score = float(base_score)
            if not analysis['has_format_error']:
                turn_eff = (MAX_TURNS - analysis['num_turns']) / MAX_TURNS
                score += turn_eff * 0.2
                if analysis['num_searches'] <= 2:
                    score += 0.1
            final_scores.append(round(score, 2))

        for traj, score in zip(trajectories, final_scores):
            traj.reward = float(score)

        print(f"  Scores: {final_scores}")

    except Exception as e:
        print(f"Judge error: {e}")
        for traj, analysis in zip(trajectories, analyses):
            traj.reward = -1.5 if analysis['has_format_error'] else 0.5

    return group


print("✅ Judge function defined with updated scoring criteria")

In [ ]:
from art.utils import iterate_dataset
import wandb

# Import centralized config
from config.training_config import WANDB_CONFIG
import config_serverless

# Initialize W&B
wandb.login(key=os.environ["WANDB_API_KEY"])

# Get serverless config
cfg = config_serverless.SERVERLESS_CONFIG

run = wandb.init(
    project=WANDB_CONFIG["project"],
    config={
        "model": cfg["base_model"],
        "groups_per_step": cfg["groups_per_step"],
        "num_epochs": cfg["num_epochs"],
        "rollouts_per_group": cfg["rollouts_per_group"],
        "learning_rate": cfg["learning_rate"],
        "max_steps": cfg["max_steps"],
        "max_turns": MAX_TURNS,
    },
)

print("🚀 Starting training loop...")
print(f"💻 Model inference: W&B Serverless")
print(f"📊 W&B Dashboard: {run.url}")
print(f"🏷️  Run: {run.name}\\n")

# Create training iterator
training_iterator = iterate_dataset(
    training_scenarios,
    groups_per_step=cfg["groups_per_step"],
    num_epochs=cfg["num_epochs"],
    initial_step=0,
)

step_count = 0

for batch in training_iterator:
    print(f"=== Step {step_count} | Epoch {batch.epoch} ===")
    
    # Create trajectory groups
    groups = [
        art.TrajectoryGroup([
            rollout(model, LegalScenarioStep(step=batch.step, scenario=scenario))
            for _ in range(cfg["rollouts_per_group"])
        ])
        for scenario in batch.items
    ]
    
    # Gather trajectories
    finished_groups = await art.gather_trajectory_groups(
        groups,
        pbar_desc="Rollouts",
        max_exceptions=cfg["rollouts_per_group"] * len(batch.items),
    )
    
    # Judge
    judged_groups = [await gemini_judge(g) for g in finished_groups]
    
    # Log metrics
    all_rewards = [t.reward for g in judged_groups for t in g.trajectories]
    wandb.log({
        "step": step_count,
        "avg_reward": sum(all_rewards) / len(all_rewards),
        "max_reward": max(all_rewards),
        "min_reward": min(all_rewards),
    })
    
    print(f"  Rewards: avg={sum(all_rewards)/len(all_rewards):.3f}")
    
    # Train
    await model.delete_checkpoints()
    await model.train(judged_groups, config=art.TrainConfig(learning_rate=cfg["learning_rate"]))
    
    print(f"✅ Step {step_count} complete\\n")
    
    step_count += 1
    if step_count >= cfg["max_steps"]:
        break

run.finish()
print("🎉 Training complete!")
print(f"📊 View results: {run.url}")